## Webscraping Genre

**Notebook beinhaltet**:
+ Webscraping von Genre1 und Genre2
+ Erweiterung des offiziellen Datensatzes

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [20]:
df3 = pd.read_csv("../songdata.csv")

In [21]:
df3

,Unnamed: 0,artist,song,link,text,Genre1,Genre2
0,0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",Pop,Rock
1,1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",Pop,Rock
2,2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,Pop,Rock
3,3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,Pop,Rock
4,4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,Pop,Rock
...,...,...,...,...,...,...,...
57645,57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...,Reggae,Pop
57646,57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...,Reggae,Pop
57647,57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...,Electronic,Pop
57648,57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...,Electronic,Pop


In [4]:
artists = list(np.unique(df3.artist))

In [6]:
len(artists)

643

In [7]:
series = pd.Series(dtype = str)
counter = 0
for element in artists:
    
    URL = f'https://www.discogs.com/search/?q={element}&type=all'
    
    html = requests.get(URL).content
    soup = BeautifulSoup(html, 'lxml')
    genre = soup.find_all("ul", class_='no_vertical facets_nav')
    for i in genre[:1]:
        genres = i.find_all('span', class_="facet_name")
        for j in genres[:2]:
            x = pd.Series(j.text, index = [element])
            series= series.append(x)
            
            series = series.groupby(level = 0).apply(lambda x: "; ".join(x.astype(str)))
    counter += 1
                
    df = pd.DataFrame({'Genre1': series})
            
        

In [8]:
df

,Genre1
'n Sync,Electronic; Pop
ABBA,Pop; Rock
Ace Of Base,Electronic; Pop
Adam Sandler,Rock; Pop
Adele,Pop; Rock
...,...
Zoegirl,Pop; Rock
Zornik,Rock; Pop
Zox,Rock; Electronic
Zucchero,Pop; Rock


### Genre Splitten

In [9]:
df[['Genre1','Genre2']] = df['Genre1'].str.split(';',expand=True)
df

,Genre1,Genre2
'n Sync,Electronic,Pop
ABBA,Pop,Rock
Ace Of Base,Electronic,Pop
Adam Sandler,Rock,Pop
Adele,Pop,Rock
...,...,...
Zoegirl,Pop,Rock
Zornik,Rock,Pop
Zox,Rock,Electronic
Zucchero,Pop,Rock


### Genre zu Dataframe hinzufügen

In [11]:
df2 = pd.read_csv("../songdata.csv")

In [46]:
del df2['Unnamed: 0']

In [12]:
df2

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [13]:
import numpy as np
d = df.Genre1
m = d.T.squeeze()
for index, value in m.items():
    
   
    df2.loc[df2['artist'].str.contains(index, case=False, na = False), 'Genre1'] = value

In [14]:
df2

,artist,song,link,text,Genre1
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",Pop
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",Pop
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,Pop
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,Pop
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,Pop
...,...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...,Reggae
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...,Reggae
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...,Electronic
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...,Electronic


In [15]:
import numpy as np
d = df.Genre2
m = d.T.squeeze()
for index, value in m.items():
    
   
    df2.loc[df2['artist'].str.contains(index, case=False, na = False), 'Genre2'] = value

In [16]:
df2

,artist,song,link,text,Genre1,Genre2
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",Pop,Rock
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",Pop,Rock
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,Pop,Rock
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,Pop,Rock
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,Pop,Rock
...,...,...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...,Reggae,Pop
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...,Reggae,Pop
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...,Electronic,Pop
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...,Electronic,Pop


In [17]:
df2.to_csv("songdata.csv")